In [ ]:
# Load Libraries.
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import pandas as pd
from datetime import datetime as dt
import time as t
import os
import re
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from urllib.parse import urljoin

In [ ]:
# Initialization. The URL we begin with, incognito mode, maximised browser window, initiation of the driver 
# and the creation of the tool SOUP for locating the information we look for
options = Options()
options.add_argument('--incognito')
options.add_argument('start-maximized')
driver = webdriver.Chrome(options = options)
driver.get("https://www.********.dk/en/**********************/")
t.sleep(5)
html = driver.page_source
soup = bs(html, features = "html.parser")


# The base url needed for updating the driver regarding the URL changes. 
base = "https://www.*******.dk"


# The information we aim to collect 
Areas = []
Sub_title = []
extra_info_list = ["Property type","Furnished","Shareable","Elevator","Balcony","Parking","Dishwasher","Washing machine",
                   "Dryer", "Deposit", "Creation Date","Move-in price", "Utilities", "Rental period", "Size", "Rooms",
                   "Monthly net rent", "Areas", "Sub_title", "Pets allowed", "Senior friendly"]


# We collect all the data in a dictionary both for efficiency and reasons of convenience.
empty_dict = {}
for name in extra_info_list:
    empty_dict[name] = []


# A function to eliminate the cookie window appearing when entering the webpage for the very first time
def click_cookie(driver, xpath):
    try:
        driver.find_element(By.XPATH, xpath)
    except NoSuchElementException:
        return False
    return True

# A function to eliminate the search agent window appearing when entering the webpage for the very first time.
def click_search_agent(driver, xpath):
    try:
        driver.find_element(By.XPATH, xpath)
    except NoSuchElementException:
        return False
    return True

# Clicking "Decline All" on the cookie preferences window
if click_cookie(driver,"//*[@id='declineButton']"):
    element = driver.find_element(By.XPATH,"//*[@id='declineButton']")
    driver.execute_script('arguments[0].scrollIntoView();', element)
    driver.execute_script('window.scrollBy(0,-200);')
    element.click()
else:
    print('No cookie preferences window')

# Closing the Search Agent window.
if click_search_agent(driver,"//*[@id='radix-:R96q:']/button"):
    element = driver.find_element(By.XPATH,"//*[@id='radix-:R96q:']/button")
    driver.execute_script('arguments[0].scrollIntoView();', element)
    driver.execute_script('window.scrollBy(0,-200);')
    element.click()
else:
    print('No search agent window')

# Roaming all pagination links by firstly checking the existence of the "Next" button 
# and then clicking it if exists, or if not, terminating the process. 
def find_pagelink(driver, xpath):
    try:
        driver.find_element(By.XPATH, xpath)
    except NoSuchElementException:
        return False
    return True

# Index used for not repeating the info collection from the promoted ads
# (in Incognito mode the ads do not change)
i = 1

# The loop populating the dictionary with all the entries available on BoligPortal.
while find_pagelink(driver,"//*[@id='app']/div[1]/div/div[1]/div/div/div[2]/div[9]/div/div/button[2]"):
    
    if i == 1:
        listings = soup.find_all("div", class_="css-krvsu4")
    else:
        listings = soup.find_all("div", class_="css-krvsu4")
        listings = listings[3:]# We ignore the first 3 promoted entries from the second pagination link onwards.

    for listing in listings:
        link = listing.find("a", href = True)
        if link :
            rel_href = link["href"]
            href = urljoin(base, rel_href)
            while True:
                try:
                    # Try to navigate to the URL using the WebDriver
                    driver.get(href)
                    t.sleep(2)  # Sleep after successful navigation
                    break  # Exit the loop if successful
                
                except WebDriverException as e:
                    print(f"Error occurred while trying to get {href}: {e}")
                    print("Retrying...")
                    t.sleep(5)  # Wait before retrying
            #driver.get(href)
            #t.sleep(2) 
            listing_html = driver.page_source
            listing_soup = bs(listing_html, 'html.parser')
            #utilities.append(listing_soup.find("span", class_ = "css-14bctuo").text.strip())
            #move_in.append(listing_soup.find("div", class_ = "css-x1kcbm").find("span", class_ = "css-14bctuo").text.strip())
            empty_dict["Sub_title"].append(listing_soup.find('h3', class_ = "css-1o5zkyw").text.strip())
            empty_dict["Areas"].append(listing_soup.find("div", class_ = "css-1gjufnd").find(class_ = "css-o9y6d5").text.strip())
            boxes = listing_soup.find_all("div", class_ = "css-1n6wxiw")# All the rows of information after you click a listing
            for box in boxes:
                text = box.find("span", class_ = "css-1td16zm").text.strip()# One-by-one all the types of information within these rows for each listing.
                if text in extra_info_list:
                    value = box.find("span", class_ = "css-1f8murc").text.strip()
                    empty_dict[text].append(value)
            if (len(empty_dict["Utilities"]) < len(empty_dict[extra_info_list[0]])):
                empty_dict["Utilities"].append(0)
            
        driver.back()
        t.sleep(2)  # Let the page reload
        html = driver.page_source
        soup = bs(html, 'html.parser')  # Update soup after returning from the listing
        
    # When the collection from listing ends and we go back to the list of listings, the search agent window may appear, hence we need to 
    # shut it down in the following fashion.
    if click_search_agent(driver,"//*[@id='radix-:R96q:']/button"):
        element = driver.find_element(By.XPATH,"//*[@id='radix-:R96q:']/button")
        driver.execute_script('arguments[0].scrollIntoView();', element)
        driver.execute_script('window.scrollBy(0,-200);')
        element.click()
        
        
    # Move to the NEXT page after collecting the information from the previous one.
    element = driver.find_element(By.XPATH,"//*[@id='app']/div[1]/div/div[1]/div/div/div[2]/div[9]/div/div/button[2]")# Find the 'NEXT' button
    driver.execute_script('arguments[0].scrollIntoView();', element)# Scroll over it.
    driver.execute_script('window.scrollBy(0,-200);')
    element.click()

    # Let the page load and inform your driver about the new url.
    t.sleep(2)
    html = driver.page_source
    soup = bs(html, 'html.parser')

    # Updating our index.
    i+=1 